In [1]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import galsim
from copy import deepcopy
import random

from syotools.models import Camera, Telescope, Spectrograph, PhotometricExposure, SpectrographicExposure
from syotools.utils.jsonunit import str_jsunit

# set paths to various directories based on the machine this code is being executed on
try:
    with initialize(version_base=None, config_path='config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']
except:
    with initialize(version_base=None, config_path='../../config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs
from mejiro.lenses import lens_util

Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.


/data/bwedig/.conda/envs/mejiro/lib/python3.10/site-packages/pysynphot/units.py:824: RuntimeWarning: divide by zero encountered in log
  return -1.085736 * N.log(arg) + ABZERO


Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.


**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-065 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-068
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


In [2]:
# pipeline_params = util.hydra_to_dict(config.pipeline)
# debugging = pipeline_params['debugging']
# # debugging = True  # TODO TEMP

# if debugging:
#     pipeline_dir = f'{config.machine.pipeline_dir}_dev'
# else:
#     pipeline_dir = config.machine.pipeline_dir

# detectable_lenses = lens_util.get_detectable_lenses(pipeline_dir, with_subhalos=False, verbose=True)

In [3]:
c, t = Camera(), Telescope()
t.set_from_json('EAC2')

Setting Telescope to:  EAC2


In [4]:
t.add_camera(c)

In [5]:
for attr in t._tracked_attributes:
    print('{}: {}'.format(attr, str_jsunit(getattr(t,attr))))

name: HWO-EAC-2
aperture: 6.0 m
temperature: 270.0 K
ota_emissivity: 0.24
diff_limit_wavelength: 500.0 nm
unobscured_fraction: 1.0


In [9]:
from astropy import units as u

assert t.aperture.unit == u.m

In [8]:
print(t.aperture)
print(type(t.aperture))

6.0 m
<class 'astropy.units.quantity.Quantity'>


In [ ]:
for attr in c._tracked_attributes:
    print('{}: {}, ({})'.format(attr, str_jsunit(getattr(c,attr))))

name: HDI
pivotwave: [ 225.  275.  336.  475.  606.  775.  850. 1260. 1600. 2220.] nm
bandnames: ['FUV', 'NUV', 'U', 'B', 'V', 'R', 'I', 'J', 'H', 'K']
channels: [[0, 1], [2, 3, 4, 5, 6], [7, 8, 9]]
fiducials: [ 400.  400. 1200.] nm
total_qe: [0.1   0.112 0.175 0.211 0.211 0.211 0.145 0.342 0.342 0.335] electron / ph
ap_corr: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 
bandpass_r: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.] 
dark_current: [0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002] electron / (pix s)
detector_rn: [2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5] electron(1/2) / pix(1/2)
sky_sigma: [24.56 26.27 23.38 22.8  22.41 22.2  22.11 22.32 22.53 22.88] 


In [22]:
c.recover('ap_corr').value

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [15]:
print(type(c.bandnames))

<class 'list'>


In [19]:
len(c.recover('dark_current'))

10

In [10]:
print(c.dark_current)
print(type(c.dark_current))

['JsonUnit', OrderedDict([('value', [0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002]), ('unit', 'electron / (pix s)')])]
<class 'list'>


In [13]:
print(c.recover('dark_current'))
print(type(c.recover('dark_current')))

[0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002] electron / (pix s)
<class 'astropy.units.quantity.Quantity'>


In [20]:
print(c.recover('dark_current').value)
print(type(c.recover('dark_current').value))

[0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002]
<class 'numpy.ndarray'>


In [14]:
dark_current = c.recover('dark_current')
print(dark_current.unit)

electron / (pix s)
